In [2]:
from rtgemlib import RTGEM
from rtgemlib import sample_from_tgem, LogLikelihood, scoreBic, mle_lambdas, LocaleLogLikelihood, get_count_duration_df, get_node_LogLikelihood, set_pcv_lambda_t, backward_neighbors_gen,\
compute_logLikelihood, set_nodes_timeseries, set_nodes_parents_counts, duration, get_parents_count_vector, forward_neighbors_gen

from tqdm.autonotebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

C:\Users\Mathilde\Anaconda35\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
def empty_nodes(nodes):
    return dict(zip(nodes, [{'timescales': {}, 'lambdas': {(): 1}}] * len(nodes)))

In [4]:
model = {'B': 
            {
            'timescales': {'A' : [[1,2], [5, 6]]},\
            'lambdas': {
                       (0,0): 10, \
                       (0,1): 1.6, \
                       (1,0): 3, \
                       (1,1) : 1
                      }
            },
            'A': {
                'timescales': {'B': [[0,1], [10,15]]},\
                'lambdas': {
                       (0,0): 1, \
                       (0,1): 4, \
                       (1,0): 5, \
                       (1,1) : 9
                      }
            }
        }



In [5]:
rtgem_model = RTGEM(model)

In [6]:
t_max = 10000

## Sampling

In [13]:
sampled_data = sample_from_tgem(rtgem_model, t_min=0, t_max=t_max)

HBox(children=(IntProgress(value=0, description='Sampled Nodes: ', max=1, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Sampling B, A:', style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='exporting data to DF', max=2, style=ProgressStyle(description_width='initial')), HTML(value='')))

In [14]:
set_pcv_lambda_t(model=rtgem_model, data=sampled_data, t_max=t_max)

In [15]:
count_duration_df = get_count_duration_df(model=rtgem_model, data=sampled_data, t_max=t_max)

In [16]:
count_duration_df

,event,pcv,lambda_t,duration,count
0,B,"(0, 0)",10.0,1.287303,10
1,B,"(0, 1)",1.6,47.214842,59
2,B,"(1, 0)",3.0,51.214842,99
3,B,"(1, 1)",1.0,9900.283014,10092
4,A,"(0, 0)",1.0,25.464509,23
5,A,"(0, 1)",4.0,3505.191253,13752
6,A,"(1, 0)",5.0,66.152066,324
7,A,"(1, 1)",9.0,6403.192171,57783


## Likelihood

In [17]:
compute_logLikelihood(count_duration_df)

64559.46066459998

In [18]:
LogLikelihood(model=rtgem_model, observed_data=sampled_data, t_max=t_max)

64559.46066459998

## Parameters learning (lambdas)

In [19]:
mle_lambdas(data=sampled_data, model=rtgem_model, t_max=t_max)

,event,pcv,lambda_t,duration,count
0,B,"(0, 0)",7.768181,1.287303,10
1,B,"(0, 1)",1.249607,47.214842,59
2,B,"(1, 0)",1.933033,51.214842,99
3,B,"(1, 1)",1.019365,9900.283014,10092
4,A,"(0, 0)",0.903218,25.464509,23
5,A,"(0, 1)",3.923324,3505.191253,13752
6,A,"(1, 0)",4.897806,66.152066,324
7,A,"(1, 1)",9.024093,6403.192171,57783


## Structure learning

### Modèle de référence

In [20]:
rtgem_model = RTGEM(empty_nodes(['A', 'B']), default_end_timescale=1)

In [21]:
rtgem_model.add_edge_operator(('A', 'A'))
rtgem_model.add_edge_operator(('A', 'B'))
rtgem_model.add_edge_operator(('B', 'A'))

In [22]:
rtgem_model.split_operator(edge=('A', 'A'), timescale=[0,1])

In [23]:
rtgem_model.extend_operator(edge=('A', 'B'))

In [24]:
sampled_data = sample_from_tgem(rtgem_model, t_min=0, t_max=10000)

HBox(children=(IntProgress(value=0, description='Sampled Nodes: ', max=1, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Sampling A, B:', style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='exporting data to DF', max=2, style=ProgressStyle(description_width='initial')), HTML(value='')))

In [25]:
t_max = 1000
data = sampled_data[sampled_data['time'] < t_max]

In [28]:
score_bic_reference = scoreBic(model=rtgem_model, observed_data=data, t_max=t_max)

### Forward Search

In [27]:
import itertools
import random
import copy

In [27]:
def empty_nodes(nodes):
    return dict(zip(nodes, [{'timescales': {}, 'lambdas': {(): 1}}] * len(nodes)))

In [33]:
model = RTGEM(empty_nodes(['A', 'B']),  default_end_timescale=1)

#### Initialisation 

In [34]:
model = set_nodes_timeseries(model, data)
model = set_nodes_parents_counts(model, model.dpd_graph.nodes, t_max)
set_pcv_lambda_t(model, data, t_max)

lambdas_count_duration_df = get_count_duration_df(data, model, t_max)

LogL = compute_logLikelihood(lambdas_count_duration_df)
log_td = np.log(t_max)

size_log_td = model.size() * log_td

score = LogL - size_log_td
local_maximum = False
nodes = list(model.dpd_graph.nodes)
possible_edges = list(itertools.product(nodes, repeat = 2))

random.shuffle(possible_edges)

In [35]:
it = 0
forward_logs = []

bic_scores_forward = []

while not local_maximum:
    #     max_ngbr_score = -np.inf
    local_maximum = True
    max_score_ngbr = -np.inf
    max_op = None
    max_args = None
    max_changed_node_cnt_drt_df = None
    max_size_log_td_ngbr = None
    max_LogL_ngbr = None
    print('iteration number: {}: scoreBIC = {}'.format(it, score))
    for ngbr_info in forward_neighbors_gen(model, data, t_max, lambdas_count_duration_df,LogL, size_log_td, log_td,\
                                           possible_edges):

        op, args, LogL_ngbr, size_log_td_ngbr, changed_node_cnt_drt_df = ngbr_info
        score_ngbr = LogL_ngbr - size_log_td_ngbr

        if score_ngbr > max_score_ngbr:
            max_score_ngbr = score_ngbr
            max_op = op
            max_args = args
            max_changed_node_cnt_drt_df = changed_node_cnt_drt_df
            max_size_log_td_ngbr = size_log_td_ngbr
            max_LogL_ngbr = LogL_ngbr
    print('max ngbr {}, args={}, max_scoreBIC = {}'.format(max_op, max_args, max_score_ngbr))
 
    if max_score_ngbr > score:
        max_op(*max_args)
        LogL = max_LogL_ngbr
        size_log_td = max_size_log_td_ngbr
        changed_node = max_changed_node_cnt_drt_df.iloc[0]['event']
        lambdas_count_duration_df = lambdas_count_duration_df[lambdas_count_duration_df['event'] != changed_node]
        lambdas_count_duration_df = pd.concat([lambdas_count_duration_df, max_changed_node_cnt_drt_df])

        local_maximum = False
        score = max_score_ngbr
        op_name = 'étendreIntervalle'

        # removes added edge from possible edges
        if max_op == model.add_edge_operator:
            possible_edges.remove(max_args[0])
        # pd.Dataframe(columns=['it', 'T_A', 'T_B', 'edges', 'max_ngbr', 'scoreBic'])
            op_name = 'ajouterArc'
        if max_op == model.split_operator:
            op_name = 'diviserIntervalle'

        forward_logs.append([it, copy.deepcopy(model.get_node_parents_timescales('A')),\
                             copy.deepcopy(model.get_node_parents_timescales('B')),\
                             list(model.dpd_graph.edges()),\
                             op_name,\
                             max_args,\
                             score])
    
    bic_scores_forward.append(score) # stores the bic score for each iteration
    
    it += 1

iteration number: 0: scoreBIC = -2013.8155105579642
max ngbr <bound method RTGEM.add_edge_operator of <rtgemlib.rtgem.RTGEM object at 0x000001BD6E7D6320>>, args=[('A', 'A')], max_scoreBIC = 4317.0729533041085
iteration number: 1: scoreBIC = 4317.0729533041085
max ngbr <bound method RTGEM.add_edge_operator of <rtgemlib.rtgem.RTGEM object at 0x000001BD6E7D6320>>, args=[('B', 'A')], max_scoreBIC = 9641.795062714784
iteration number: 2: scoreBIC = 9641.795062714784
max ngbr <bound method RTGEM.add_edge_operator of <rtgemlib.rtgem.RTGEM object at 0x000001BD6E7D6320>>, args=[('B', 'B')], max_scoreBIC = 13293.528892231472
iteration number: 3: scoreBIC = 13293.528892231472
max ngbr <bound method RTGEM.extend_operator of <rtgemlib.rtgem.RTGEM object at 0x000001BD6E7D6320>>, args=[('B', 'B')], max_scoreBIC = 16938.69582156432
iteration number: 4: scoreBIC = 16938.69582156432
max ngbr <bound method RTGEM.extend_operator of <rtgemlib.rtgem.RTGEM object at 0x000001BD6E7D6320>>, args=[('B', 'B')], m

KeyboardInterrupt: 

In [36]:
model = set_nodes_timeseries(model, data)
model = set_nodes_parents_counts(model, model.dpd_graph.nodes, t_max)

In [37]:
mle_lambdas(model, data, t_max)

,event,pcv,lambda_t,duration,count
0,A,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",0.000000,0.256949,0
1,A,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 1)",0.000000,0.774529,0
2,A,"(0, 0, 0, 0, 0, 0, 0, 0, 1, 0)",0.000000,0.080890,0
3,A,"(0, 0, 0, 0, 0, 0, 0, 0, 1, 1)",0.000000,47.506103,0
4,A,"(0, 0, 0, 0, 0, 0, 0, 1, 0, 0)",0.000000,0.000000,0
5,A,"(0, 0, 0, 0, 0, 0, 0, 1, 0, 1)",0.000000,5.053514,0
6,A,"(0, 0, 0, 0, 0, 0, 0, 1, 1, 0)",0.000000,0.218888,0
7,A,"(0, 0, 0, 0, 0, 0, 0, 1, 1, 1)",0.000000,51.247001,0
8,A,"(0, 0, 0, 0, 0, 0, 1, 0, 0, 0)",0.000000,0.000000,0
9,A,"(0, 0, 0, 0, 0, 0, 1, 0, 0, 1)",0.000000,4.927416,0


In [38]:
scoreBic(model, data, t_max)

16681.693106105904

In [39]:
# keep forward result in memory
forward_model = copy.deepcopy(model)

## BackwardSearch(Forward)

In [34]:
model = set_nodes_timeseries(model, data)
model = set_nodes_parents_counts(model, model.dpd_graph.nodes, t_max)
set_pcv_lambda_t(model, data, t_max)

lambdas_count_duration_df = get_count_duration_df(data, model, t_max)

LogL = compute_logLikelihood(lambdas_count_duration_df)
log_td = np.log(t_max)

size_log_td = model.size() * log_td

score = LogL - size_log_td
local_maximum = False

In [35]:
score

-1897.0277331403577

In [36]:
it = 0
backward_logs = []
local_maximum = False

bic_scores_backward = []

while not local_maximum:
    #     max_ngbr_score = -np.inf
    local_maximum = True
    max_score_ngbr = -np.inf
    max_op = None
    max_args = None
    max_changed_node_cnt_drt_df = None
    max_size_log_td_ngbr = None
    max_LogL_ngbr = None
    print('iteration number: {}: scoreBIC = {}'.format(it, score))
    for ngbr_info in backward_neighbors_gen(model, data, t_max, lambdas_count_duration_df,LogL, size_log_td, log_td,):

        op, args, LogL_ngbr, size_log_td_ngbr, changed_node_cnt_drt_df = ngbr_info
        score_ngbr = LogL_ngbr - size_log_td_ngbr

#         if score_ngbr > max_score_ngbr:
        max_score_ngbr = score_ngbr
        max_op = op
        max_args = args
        max_changed_node_cnt_drt_df = changed_node_cnt_drt_df
        max_size_log_td_ngbr = size_log_td_ngbr
        max_LogL_ngbr = LogL_ngbr

        if max_score_ngbr > score:
            print('max ngbr {}, args={}, max_scoreBIC = {}'.format(max_op, max_args, max_score_ngbr))

            max_op(*max_args)
            LogL = max_LogL_ngbr
            size_log_td = max_size_log_td_ngbr
            changed_node = max_changed_node_cnt_drt_df.iloc[0]['event']
            lambdas_count_duration_df = lambdas_count_duration_df[lambdas_count_duration_df['event'] != changed_node]
            lambdas_count_duration_df = pd.concat([lambdas_count_duration_df, max_changed_node_cnt_drt_df])

            local_maximum = False
            score = max_score_ngbr
            op_name = 'supprimerArc'

            if max_op == model.inverse_extend_operator:
                op_name = 'reduireIntervalle'
            # pd.Dataframe(columns=['it', 'T_A', 'T_B', 'edges', 'max_ngbr', 'scoreBic'])
            if max_op == model.inverse_split_operator:
                op_name = 'FusionnerIntervalle'

            backward_logs.append([it, copy.deepcopy(model.get_node_parents_timescales('A')),\
                                 copy.deepcopy(model.get_node_parents_timescales('B')),\
                                 list(model.dpd_graph.edges()),\
                                 op_name,\
                                 max_args,\
                                 score])
            
            break
    
    bic_scores_backward.append(score)
    
    it += 1

iteration number: 0: scoreBIC = -1897.0277331403577
max ngbr <bound method RTGEM.inverse_add_edge_operator of <rtgemlib.rtgem.RTGEM object at 0x00000160F0256898>>, args=[('A', 'B')], max_scoreBIC = -160.12474173196279
iteration number: 1: scoreBIC = -160.12474173196279
max ngbr <bound method RTGEM.inverse_extend_operator of <rtgemlib.rtgem.RTGEM object at 0x00000160F0256898>>, args=[('A', 'A')], max_scoreBIC = 1608.1398232544834
iteration number: 2: scoreBIC = 1608.1398232544834
max ngbr <bound method RTGEM.inverse_extend_operator of <rtgemlib.rtgem.RTGEM object at 0x00000160F0256898>>, args=[('A', 'A')], max_scoreBIC = 2492.0021361774925
iteration number: 3: scoreBIC = 2492.0021361774925
max ngbr <bound method RTGEM.inverse_extend_operator of <rtgemlib.rtgem.RTGEM object at 0x00000160F0256898>>, args=[('A', 'A')], max_scoreBIC = 2933.927177722613
iteration number: 4: scoreBIC = 2933.927177722613
max ngbr <bound method RTGEM.inverse_extend_operator of <rtgemlib.rtgem.RTGEM object at 0x

In [ ]:

# Fixing random state for reproducibility
np.random.seed(19680801)

# create random data
xdata = np.random.random([2, 10])


In [ ]:
xdata

In [ ]:

# split the data into two parts
xdata1 = xdata[0, :]
xdata2 = xdata[1, :]

# sort the data so it makes clean curves
xdata1.sort()
xdata2.sort()

In [ ]:
# create some y data points
ydata1 = xdata1 ** 2
ydata2 = 1 - xdata2 ** 3

# plot the data
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(xdata1, ydata1, 'r', xdata2, ydata2, 'b')

In [ ]:




# create the events marking the x data points
xevents1 = EventCollection(xdata1, color=[1, 0, 0], linelength=0.05)
xevents2 = EventCollection(xdata2, color=[0, 0, 1], linelength=0.05)

# create the events marking the y data points
yevents1 = EventCollection(ydata1, color=[1, 0, 0], linelength=0.05,
                           orientation='vertical')
yevents2 = EventCollection(ydata2, color=[0, 0, 1], linelength=0.05,
                           orientation='vertical')

# add the events to the axis
ax.add_collection(xevents1)
ax.add_collection(xevents2)
ax.add_collection(yevents1)
ax.add_collection(yevents2)

# set the limits
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

ax.set_title('line plot with data points')

# display the plot
plt.show()